In [ ]:
# DOCUMENTAZIONE https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html

## CONFIGURAZIONE

In [ ]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 5.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from bertopic import BERTopic
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

/usr/local/lib/python3.12/dist-packages/hdbscan/plots.py:448: SyntaxWarning: invalid escape sequence '\l'
  axis.set_ylabel('$\lambda$ value')
/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importing dataset
df_ozempic = pd.read_csv("/content/drive/MyDrive/Datasets/Scrapings/ozempic_dataset.csv")
df_mounjaro = pd.read_csv("/content/drive/MyDrive/Datasets/Scrapings/mounjaro_dataset.csv")
df_rybelsus = pd.read_csv("/content/drive/MyDrive/Datasets/Scrapings/rybelsus_dataset.csv")
df_trulicity = pd.read_csv("/content/drive/MyDrive/Datasets/Scrapings/trulicity_dataset.csv")
df_wegovy = pd.read_csv("/content/drive/MyDrive/Datasets/Scrapings/wegovy_dataset.csv")

In [ ]:
df_ozempic.shape, df_mounjaro.shape, df_rybelsus.shape, df_trulicity.shape, df_wegovy.shape

((4330, 10), (5322, 10), (1232, 10), (887, 10), (4454, 10))

In [ ]:
df_ozempic['selftext']

,selftext
0,"Many of you probably already know this, but it..."
1,Someone I know has gained 6 lb in two months o...
2,"I'm 44, type 2 diabetes, one heart attack, sli..."
3,NaN
4,I’m under 160 pounds for the first time since ...
...,...
4325,Hey I'm asking this with the request that ever...
4326,I was diagnosed last year with type 2 diabetes...
4327,Health Canada has approved Ozempic to reduce t...
4328,My wife has type II diabetes and was successfu...


Now we apply the semantic analysis on these datasets

## OZEMPIC

In [ ]:
# -------- OZEMPIC ----------

text_ozempic = df_ozempic['selftext'].str.lower()

# Remove very basic noise
text_ozempic = text_ozempic.str.replace('\n', ' ', regex=False)  # Remove line breaks
text_ozempic = text_ozempic.str.replace(r'\s+', ' ', regex=True)  # Remove extra spaces

# Remove empty/null texts
text_ozempic = text_ozempic.dropna()
text_ozempic = text_ozempic[text_ozempic.str.len() > 10]  # Keep only texts with more than 10 characters

In [ ]:
# trovare il best pca ----------------

# # Supponiamo di avere già i testi
# text.dropna().reset_index(drop=True, inplace=True)

# # 1) Embedding
# embedding_model = SentenceTransformer("all-mpnet-base-v2")
# embeddings = embedding_model.encode(text, show_progress_bar=True)

# # ---------------------------
# # STEP 1: PCA con 90% varianza
# # ---------------------------
# pca_full = PCA().fit(embeddings)
# cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
# n_components = np.argmax(cumulative_variance >= 0.90) + 1   # numero componenti per 90%

# print(f"PCA scelto: {n_components} componenti per spiegare il 90% della varianza")

# dim_model = PCA(n_components=n_components)

# # Riduzione degli embedding
# reduced_embeddings = dim_model.fit_transform(embeddings)

In [ ]:
from bertopic import BERTopic
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# configuring the vectorizer
embedding_model = SentenceTransformer("all-mpnet-base-v2")
representation_model = KeyBERTInspired()
# cluster_model = KMeans(n_clusters=15) le metriche del kmeans non portano a nessun risultato, utilizzo hdbscan
dim_model = PCA(n_components=174) #mantengo il 90% della varianza

vectorizer_model = CountVectorizer(
    ngram_range=(1, 2),   # considera unigrammi e bigrammi
    stop_words= "english",
    #min_df=0.05,             # ignora termini che appaiono in meno di 5 documenti
    #max_df=0.9               # ignora termini che appaiono in più del 90% dei documenti
)

kmeans_model = KMeans(n_clusters=15, random_state=42)

topic_model = BERTopic(
    embedding_model = embedding_model,
    umap_model = dim_model,
    hdbscan_model = kmeans_model,
    representation_model = representation_model,
    vectorizer_model = vectorizer_model,
    top_n_words=15,
    nr_topics=None,
    calculate_probabilities=True,
    verbose=True
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
topics, probs = topic_model.fit_transform(text_ozempic)

2025-09-21 18:13:36,147 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/127 [00:00<?, ?it/s]

2025-09-21 18:14:06,738 - BERTopic - Embedding - Completed ✓
2025-09-21 18:14:06,741 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-21 18:14:09,216 - BERTopic - Dimensionality - Completed ✓
2025-09-21 18:14:09,229 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-21 18:14:09,623 - BERTopic - Cluster - Completed ✓
2025-09-21 18:14:09,631 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-21 18:14:11,178 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model.get_topic(1)

[('medication', np.float32(0.3860467)),
 ('taking ozempic', np.float32(0.33120486)),
 ('started taking', np.float32(0.30974597)),
 ('1mg', np.float32(0.30870885)),
 ('weight', np.float32(0.30104542)),
 ('25mg', np.float32(0.27701753)),
 ('injection', np.float32(0.25027636)),
 ('nausea', np.float32(0.24911135)),
 ('food', np.float32(0.23080152)),
 ('doctor', np.float32(0.22988182))]

In [ ]:
topic_model.get_topic_info()['Name'][2]

'2_taking ozempic_ozempic weight_obesity_lose weight'

In [ ]:
info_ozempic = topic_model.get_topic_info()

In [ ]:
info_ozempic.to_csv("/content/drive/MyDrive/Datasets/semantic_analysis_ozempic.csv")

In [ ]:
info_ozempic

## MOUNJARO

In [ ]:
# -------- MOUNJARO ----------

text_mounjaro = df_mounjaro['selftext'].str.lower()

# Remove very basic noise
text_mounjaro = text_mounjaro.str.replace('\n', ' ', regex=False)  # Remove line breaks
text_mounjaro = text_mounjaro.str.replace(r'\s+', ' ', regex=True)  # Remove extra spaces

# Remove empty/null texts
text_mounjaro = text_mounjaro.dropna()
text_mounjaro = text_mounjaro[text_mounjaro.str.len() > 10]  # Keep only texts with more than 10 characters

In [ ]:
# trovare il best pca ----------------

text = text_mounjaro

# # Supponiamo di avere già i testi
text = text.dropna().reset_index(drop=True)

# # 1) Embedding
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(text, show_progress_bar=True)

# # ---------------------------
# # STEP 1: PCA con 90% varianza
# # ---------------------------
pca_full = PCA().fit(embeddings)
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
n_components = np.argmax(cumulative_variance >= 0.90) + 1   # numero componenti per 90%

print(f"PCA scelto: {n_components} componenti per spiegare il 90% della varianza")

dim_model = PCA(n_components=n_components)

# # Riduzione degli embedding
reduced_embeddings = dim_model.fit_transform(embeddings)

Batches:   0%|          | 0/161 [00:00<?, ?it/s]

PCA scelto: 171 componenti per spiegare il 90% della varianza


In [ ]:
# classe mounjaro
from bertopic import BERTopic
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# configuring the vectorizer
embedding_model = SentenceTransformer("all-mpnet-base-v2")
representation_model = KeyBERTInspired()
# cluster_model = KMeans(n_clusters=15) le metriche del kmeans non portano a nessun risultato, utilizzo hdbscan
dim_model_mounjaro = PCA(n_components=171) #mantengo il 90% della varianza

vectorizer_model = CountVectorizer(
    ngram_range=(1, 2),   # considera unigrammi e bigrammi
    stop_words= "english",
    #min_df=0.05,             # ignora termini che appaiono in meno di 5 documenti
    #max_df=0.9               # ignora termini che appaiono in più del 90% dei documenti
)

kmeans_model = KMeans(n_clusters=15, random_state=42)

topic_model_mounjaro = BERTopic(
    embedding_model = embedding_model,
    umap_model = dim_model,
    hdbscan_model = kmeans_model,
    representation_model = representation_model,
    vectorizer_model = vectorizer_model,
    top_n_words=15,
    nr_topics=None,
    calculate_probabilities=True,
    verbose=True
)

In [ ]:
topics, probs = topic_model_mounjaro.fit_transform(text_mounjaro)

2025-09-21 18:14:55,975 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/161 [00:00<?, ?it/s]

2025-09-21 18:15:37,905 - BERTopic - Embedding - Completed ✓
2025-09-21 18:15:37,909 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-21 18:15:40,726 - BERTopic - Dimensionality - Completed ✓
2025-09-21 18:15:40,734 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-21 18:15:40,936 - BERTopic - Cluster - Completed ✓
2025-09-21 18:15:40,945 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-21 18:15:43,504 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model_mounjaro.get_topic_info().to_csv("/content/drive/MyDrive/Datasets/semantic_analysis_mounjaro.csv")

## RYBELSUS

In [ ]:
# -------- RYBELSUS ----------

text_rybelsus = df_rybelsus['selftext'].str.lower()

# Remove very basic noise
text_rybelsus = text_rybelsus.str.replace('\n', ' ', regex=False)  # Remove line breaks
text_rybelsus = text_rybelsus.str.replace(r'\s+', ' ', regex=True)  # Remove extra spaces

# Remove empty/null texts
text_rybelsus = text_rybelsus.dropna()
text_rybelsus = text_rybelsus[text_rybelsus.str.len() > 10]  # Keep only texts with more than 10 characters

In [ ]:
# trovare il best pca ----------------

text = text_rybelsus

# # Supponiamo di avere già i testi
text = text.dropna().reset_index(drop=True)

# # 1) Embedding
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(text, show_progress_bar=True)

# # ---------------------------
# # STEP 1: PCA con 90% varianza
# # ---------------------------
pca_full = PCA().fit(embeddings)
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
n_components = np.argmax(cumulative_variance >= 0.90) + 1   # numero componenti per 90%

print(f"PCA scelto: {n_components} componenti per spiegare il 90% della varianza")

dim_model = PCA(n_components=n_components)

# # Riduzione degli embedding
reduced_embeddings = dim_model.fit_transform(embeddings)

Batches:   0%|          | 0/37 [00:00<?, ?it/s]

PCA scelto: 137 componenti per spiegare il 90% della varianza


In [ ]:
# classe rybelsus
from bertopic import BERTopic
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# configuring the vectorizer
embedding_model = SentenceTransformer("all-mpnet-base-v2")
representation_model = KeyBERTInspired()
# cluster_model = KMeans(n_clusters=15) le metriche del kmeans non portano a nessun risultato, utilizzo hdbscan
dim_model_rybelsus = PCA(n_components=137) #mantengo il 90% della varianza

vectorizer_model = CountVectorizer(
    ngram_range=(1, 2),   # considera unigrammi e bigrammi
    stop_words= "english",
    #min_df=0.05,             # ignora termini che appaiono in meno di 5 documenti
    #max_df=0.9               # ignora termini che appaiono in più del 90% dei documenti
)

kmeans_model = KMeans(n_clusters=15, random_state=42)

topic_model_rybelsus = BERTopic(
    embedding_model = embedding_model,
    umap_model = dim_model,
    hdbscan_model = kmeans_model,
    representation_model = representation_model,
    vectorizer_model = vectorizer_model,
    top_n_words=15,
    nr_topics=None,
    calculate_probabilities=True,
    verbose=True
)

In [ ]:
topics, probs = topic_model_rybelsus.fit_transform(text_rybelsus)

2025-09-21 18:15:59,025 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/37 [00:00<?, ?it/s]

2025-09-21 18:16:09,049 - BERTopic - Embedding - Completed ✓
2025-09-21 18:16:09,051 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-21 18:16:10,270 - BERTopic - Dimensionality - Completed ✓
2025-09-21 18:16:10,274 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-21 18:16:10,362 - BERTopic - Cluster - Completed ✓
2025-09-21 18:16:10,369 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-21 18:16:11,269 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model_rybelsus.get_topic_info().to_csv("/content/drive/MyDrive/Datasets/rybelsus_analysis_ozempic.csv")

## TRULICITY

In [ ]:
# -------- TRULICITY ----------

text_trulicity = df_trulicity['selftext'].str.lower()

# Remove very basic noise
text_trulicity = text_trulicity.str.replace('\n', ' ', regex=False)  # Remove line breaks
text_trulicity = text_trulicity.str.replace(r'\s+', ' ', regex=True)  # Remove extra spaces

# Remove empty/null texts
text_trulicity = text_trulicity.dropna()
text_trulicity = text_trulicity[text_trulicity.str.len() > 10]  # Keep only texts with more than 10 characters

In [ ]:
# trovare il best pca ----------------

text = text_trulicity

# # Supponiamo di avere già i testi
text = text.dropna().reset_index(drop=True)

# # 1) Embedding
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(text, show_progress_bar=True)

# # ---------------------------
# # STEP 1: PCA con 90% varianza
# # ---------------------------
pca_full = PCA().fit(embeddings)
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
n_components = np.argmax(cumulative_variance >= 0.90) + 1   # numero componenti per 90%

print(f"PCA scelto: {n_components} componenti per spiegare il 90% della varianza")

dim_model = PCA(n_components=n_components)

# # Riduzione degli embedding
reduced_embeddings = dim_model.fit_transform(embeddings)

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

PCA scelto: 131 componenti per spiegare il 90% della varianza


In [ ]:
# classe trulicity
from bertopic import BERTopic
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# configuring the vectorizer
embedding_model = SentenceTransformer("all-mpnet-base-v2")
representation_model = KeyBERTInspired()
# cluster_model = KMeans(n_clusters=15) le metriche del kmeans non portano a nessun risultato, utilizzo hdbscan
dim_model_trulicity = PCA(n_components=131) #mantengo il 90% della varianza

vectorizer_model = CountVectorizer(
    ngram_range=(1, 2),   # considera unigrammi e bigrammi
    stop_words= "english",
    #min_df=0.05,             # ignora termini che appaiono in meno di 5 documenti
    #max_df=0.9               # ignora termini che appaiono in più del 90% dei documenti
)

kmeans_model = KMeans(n_clusters=15, random_state=42)

topic_model_trulicity = BERTopic(
    embedding_model = embedding_model,
    umap_model = dim_model,
    hdbscan_model = kmeans_model,
    representation_model = representation_model,
    vectorizer_model = vectorizer_model,
    top_n_words=15,
    nr_topics=None,
    calculate_probabilities=True,
    verbose=True
)

In [ ]:
topics, probs = topic_model_trulicity.fit_transform(text_trulicity)

2025-09-21 18:16:24,374 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

2025-09-21 18:16:32,173 - BERTopic - Embedding - Completed ✓
2025-09-21 18:16:32,174 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-21 18:16:32,588 - BERTopic - Dimensionality - Completed ✓
2025-09-21 18:16:32,597 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-21 18:16:32,632 - BERTopic - Cluster - Completed ✓
2025-09-21 18:16:32,643 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-21 18:16:33,386 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model_trulicity.get_topic_info().to_csv("/content/drive/MyDrive/Datasets/semantic_analysis_trulicity.csv")

##

## WEGOVY

In [ ]:
# -------- WEGOVY ----------

text_wegovy = df_wegovy['selftext'].str.lower()

# Remove very basic noise
text_wegovy = text_wegovy.str.replace('\n', ' ', regex=False)  # Remove line breaks
text_wegovy = text_wegovy.str.replace(r'\s+', ' ', regex=True)  # Remove extra spaces

# Remove empty/null texts
text_wegovy = text_wegovy.dropna()
text_wegovy = text_wegovy[text_wegovy.str.len() > 10]  # Keep only texts with more than 10 characters

In [ ]:
# trovare il best pca ----------------

text = text_wegovy

# # Supponiamo di avere già i testi
text = text.dropna().reset_index(drop=True)

# # 1) Embedding
embedding_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedding_model.encode(text, show_progress_bar=True)

# # ---------------------------
# # STEP 1: PCA con 90% varianza
# # ---------------------------
pca_full = PCA().fit(embeddings)
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
n_components = np.argmax(cumulative_variance >= 0.90) + 1   # numero componenti per 90%

print(f"PCA scelto: {n_components} componenti per spiegare il 90% della varianza")

dim_model = PCA(n_components=n_components)

# # Riduzione degli embedding
reduced_embeddings = dim_model.fit_transform(embeddings)

Batches:   0%|          | 0/134 [00:00<?, ?it/s]

PCA scelto: 166 componenti per spiegare il 90% della varianza


In [ ]:
# classe wegovy
from bertopic import BERTopic
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# configuring the vectorizer
embedding_model = SentenceTransformer("all-mpnet-base-v2")
representation_model = KeyBERTInspired()
# cluster_model = KMeans(n_clusters=15) le metriche del kmeans non portano a nessun risultato, utilizzo hdbscan
dim_model_wegovy = PCA(n_components=166) #mantengo il 90% della varianza

vectorizer_model = CountVectorizer(
    ngram_range=(1, 2),   # considera unigrammi e bigrammi
    stop_words= "english",
    #min_df=0.05,             # ignora termini che appaiono in meno di 5 documenti
    #max_df=0.9               # ignora termini che appaiono in più del 90% dei documenti
)

kmeans_model = KMeans(n_clusters=15, random_state=42)

topic_model_wegovy = BERTopic(
    embedding_model = embedding_model,
    umap_model = dim_model,
    hdbscan_model = kmeans_model,
    representation_model = representation_model,
    vectorizer_model = vectorizer_model,
    top_n_words=15,
    nr_topics=None,
    calculate_probabilities=True,
    verbose=True
)

In [ ]:
topics, probs = topic_model_wegovy.fit_transform(text_wegovy)

2025-09-21 18:17:10,028 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/134 [00:00<?, ?it/s]

2025-09-21 18:17:42,978 - BERTopic - Embedding - Completed ✓
2025-09-21 18:17:42,979 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-21 18:17:43,617 - BERTopic - Dimensionality - Completed ✓
2025-09-21 18:17:43,629 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-21 18:17:43,779 - BERTopic - Cluster - Completed ✓
2025-09-21 18:17:43,784 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-21 18:17:45,212 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model_wegovy.get_topic_info().to_csv("/content/drive/MyDrive/Datasets/semantic_analysis_wegovy.csv")